# Codes for Final Submission

In [1]:
### import module
import pandas as pd
import requests as rq
import numpy as np
from bs4 import BeautifulSoup as bsp
import time
import re
import ast
import math

## Scraping the BoxOffice Rankings
## And save them to seperate CSV files in the current dirrectory

### Notice: 
- Following web-scraping part would take a long time to run
- If you have the raw_final_data.csv in the current working folder, you could directly skip to analysis & visulization part

In [ ]:
## get the rank and link to each individual movie for top 100 sales in China/NA/Global
for area in ["china","global","NorthAmerica"]:
    with open('mtime_data_'+area+'.txt', mode='w') as file:
        rank=1
        for page in range(10):
            parameters={"area":area,
                       "type":"MovieRankingHistory",
                       "category":"all",
                       "page":f'{page}',
                       "display":"list",
                       "timestamp":f'{time.time()}',
                       "version":"07bb781100018dd58eafc3b35d42686804c6df8d",
                       "dataType":"json"}
            try:
                MtimeRank = rq.get("http://movie.mtime.com/boxoffice/",params=parameters,timeout=10)
            except:
                print(f'Page {page} of area {area} for the MTime Boxoffice connection timeout')
                continue
                
            if MtimeRank.status_code!=200:
                print(f'Page {page} of area {area} for the MTime Boxoffice failed connection.')
                continue

            soup=bsp(MtimeRank.json()['html'])
            basic_info=soup.find_all("div",{"class":"txtbox"})
        
            for info in basic_info:
                for i in info.contents:
                    if i.name !="b":
                        if i.name == 'h3':
                            ## creating a personal split symbol "--;" for convenience
                            file.write('{}--;'.format(rank))
                            id_name=str(i.contents)
                            file.write('{}--;'.format(re.search("http.+[0-9]+\"",id_name).group(0)[:-1]))
                            file.write('{}--;'.format(re.search("\>.+\<",id_name).group(0)[1:-1]))
                        else:
                            file.write('{}--;'.format(i.text))
                for j in list(info.next_siblings):
                    for k in j.contents:
                        file.write('{}--;'.format(k.text))
                file.write('\n')
                # increase the rank and go to next moive
                rank+=1
        file.flush()

#### notice that sales in china is in CNY while its USD for global & NA
#### see if crutial information has been scraped

In [ ]:
## see if crutial information has been scraped 
for area in ["china","global","NorthAmerica"]:
    with open('mtime_data_'+area+'.txt', mode='r') as file:
        contents=file.read()
    assert len(re.findall("http",contents))==100
    ## check scraped information for a movie
    basic_info=[]
    info_size=[]
    with open('mtime_data_'+area+'.txt', mode='r') as file:
        for line in file:
            basic=line.strip("\n--;").split("--;")
            basic_info.append(basic)
            info_size.append(len(basic))
    # output a sample basic information
    for i in basic_info:
        if len(i)==max(info_size):
            output=i
        break
    print(output)

### enter the page of each individual movie in the ranking list

In [ ]:
#### for each of three top 100 sales
data=[]
area=["china","global","NorthAmerica"]
for i in area:
    with open("mtime_data_"+i+".txt",mode='r') as file:
        for line in file:
            data.append(i+' '+re.sub(r'(\xa0)|\n','',line))
### basic data frame
movie_name=[]
movie_eng=[]
link=[]
mtime_rate=[]
rate_users=[]
box_office=[]
BoxOfficeDetail=[]
ReleaseDate=[]
BoxOfficeRankCN=[]
BoxOfficeRankG=[]
BoxOfficeRankNA=[]
for i in data:
    infor=i.split(r"--;")
    movie_name.append(infor[2])
    movie_eng.append(infor[3])
    link.append(infor[1])
    
    if "首" in infor[4]:
        BoxOfficeDetail.append(infor[4])
    else:
        BoxOfficeDetail.append(np.NaN)
        
    ReleaseDate.append(re.search(r"\d+年\d+月\d+日\w*上映",i).group().replace("上映",""))
    
    
    sale_rate_txt=re.search(r"--;\d\.\d--;\d+人评分--;.+--;",i).group()
    sale_rate=re.sub(r"累计人次.+","",sale_rate_txt).split("--;")
    mtime_rate.append(sale_rate[1])
    rate_users.append(sale_rate[2].replace("人评分",""))
    
    if "china" in infor[0]:
        BoxOfficeRankCN.append(infor[0].replace("china ",""))
        BoxOfficeRankG.append(np.NaN)
        BoxOfficeRankNA.append(np.NaN)
        box_office.append(sale_rate[-2]+"CNY")
    elif "global" in infor[0]:
        BoxOfficeRankG.append(infor[0].replace("global ",""))
        box_office.append(sale_rate[-2]+"USD")
        BoxOfficeRankCN.append(np.NaN)
        BoxOfficeRankNA.append(np.NaN)
    elif "NorthAmerica" in infor[0]:
        BoxOfficeRankNA.append(infor[0].replace("NorthAmerica ",""))
        box_office.append(sale_rate[-2]+"USD")
        BoxOfficeRankG.append(np.NaN)
        BoxOfficeRankCN.append(np.NaN)
dat={"Name":movie_name,"NameEng":movie_eng,"MtimePage":link,"ReleaseDate":ReleaseDate,"BoxOffice":box_office,
     "BoxOfficeDetail":BoxOfficeDetail,"BoxOfficeRankCN":BoxOfficeRankCN,"BoxOfficeRankNA":BoxOfficeRankNA,
     "BoxOfficeRankG":BoxOfficeRankG,"MtimeRating":mtime_rate,
    "NumberofReviewers":rate_users}
dt=pd.DataFrame.from_dict(dat)
dt.head()

### There certainly will be overlapping among three ranking lists

In [ ]:
## fill in rankings
for i in data:
    infor=i.split(r"--;")
    if "china" in infor[0]:
        dt.loc[dt.Name==infor[2],"BoxOfficeRankCN"]=infor[0].replace("china ","")
    elif "global" in infor[0]:
        dt.loc[dt.Name==infor[2],"BoxOfficeRankG"]=infor[0].replace("global ","")
    elif "NorthAmerica" in infor[0]:
        dt.loc[dt.Name==infor[2],"BoxOfficeRankNA"]=infor[0].replace("NorthAmerica ","")
## fill in Global sales
dt_global=dt.iloc[100:200,[0,4]]
dt_global=dt_global.rename(columns={'BoxOffice':'BoxOfficeGlobal'})
dt_merged=pd.merge(dt,dt_global,how="left")

### Get Runtime, Genre(catergory), FilmFormat for each movie

In [ ]:
# more basic information for each movie
Runtime=[]
Genre=[]
FilmFormat=[]
for i in dt_merged.MtimePage:
    try:
        detailpage=rq.get(i,timeout=10)
    except:
        print(f'{i} Time out')
    if detailpage.status_code==200:
        soup=bsp(detailpage.text,"lxml")
    else:
        print(f'{i} connection failed')
        continue

    ## Runtime, Genre(catergory), FilmFormat
    more_infor=soup.body.find_all("div",{"class":"otherbox"})
    if more_infor:
        more_infor_list=more_infor[0].text.split("-")
        try:
            Runtime.append(more_infor_list[0].strip())
        except:
            Runtime.append(np.NaN)
        try:
            Genre.append(more_infor_list[1].strip())
        except:
            Genre.append(np.NaN)
        try:
            FilmFormat.append(more_infor_list[3].strip())
        except:
            FilmFormat.append(np.NaN)
    else:
        Runtime.append(np.NaN)
        Genre.append(np.NaN)
        FilmFormat.append(np.NaN)
dt_merged["Runtime"]=Runtime
dt_merged["Genre"]=Genre
dt_merged["FilmFormat"]=FilmFormat

### Get actor names and staff members including writers, directors....

In [ ]:
## actor/ staff member
## 演、职员
actor_staff=[]
for i in dt_merged.MtimePage:
    try:
        detailpage=rq.get(i+"/fullcredits.html",timeout=10)
    except:
        print(f'{i} Time out')
    if detailpage.status_code==200:
        soup=bsp(detailpage.text,"lxml")
    else:
        print(f'{i} connection failed')
        continue

    soup=bsp(detailpage.text,"lxml")
    actor_sec=soup.body.find_all("div",{"class":"actor_tit"})
    people={}
    actor_name=[]
    for i in actor_sec:
        if i.h4:
            actor=i.h4.text.strip()
        else:
            actor_name.append(i.text.strip())
    people[actor]=actor_name 
    ## Staff member
    staff_sec=soup.body.find_all("div",{"class":"credits_r"})[0].find_all("div",{"class":"credits_list"})
    for i in staff_sec:
        title=(i.h4.text.strip())
        member_list=[]
        member=i.find_all('p')
        if member:
            for p in member:
                member_list.append(p.text.strip())
        people[title]=member_list
    actor_staff.append(people)
dt_merged["ActorAndStaff"]=actor_staff

### Get winning awards for each movie

In [ ]:
awards_detail=[]
awards_sum=[]
## add awards
## 奖项
for i in dt_merged.MtimePage:
    try:
        detailpage=rq.get(i+"/awards.html",timeout=10)
    except:
        print(f'{i} Time out')
    if detailpage.status_code==200:
        soup=bsp(detailpage.text,"lxml")
    else:
        print(f'{i} connection failed')
        continue
        
    award_name=soup.body.find_all("h3")
    award_detail=soup.body.find_all("dl")
    if award_name:
        aw_names=[]
        for i in award_name:
            if i.b:
                aw_names.append(i.text.strip())
            else:
                award_summary=i.text.strip()
            
        for i in range(len(award_detail)):
            if 'id' in award_detail[i].attrs:
                if award_detail[i]['id']=="awardSlidesItems":
                    position=i+1
                    break
        details=award_detail[position:]
        assert len(details)==len(aw_names)
        
        ## for each award, get detail
        award_dict=dict()
        for i in range(len(aw_names)):
            summary=re.search(r'奖 \(.+',aw_names[i])
            if summary:
                summary=summary.group().replace(r"奖 ","")
            name=re.search(r'\w+奖',aw_names[i])
            if name:
                name=name.group()
                award_dict[name]=[summary,re.sub(r'·\w+-','#',details[i].text.strip())]
            
        awards_sum.append([award_summary,aw_names])
        awards_detail.append(award_dict) 
    else:
        award_summary=np.NaN
        awards_sum.append(np.NaN)
        awards_detail.append(np.NaN) 
dt_merged["AwardSummary"]=awards_sum
dt_merged["AwardDetail"]=awards_detail

### Get Companies & Distributors for each movie

In [ ]:
## add companies
## 公司
produce_company=[]
distribute_company=[]
for i in dt_merged.MtimePage:
    try:
        detailpage=rq.get(i+"/details.html",timeout=10)
    except:
        print(f'{i} Time out')
    if detailpage.status_code==200:
        soup=bsp(detailpage.text,"lxml")
    else:
        print(f'{i} connection failed')
        continue
    
    soup=bsp(detailpage.text,"lxml")
    company_sec=soup.body.find_all("dl",{"id":"companyRegion"})
    if company_sec:
        company_detail=company_sec[0].find_all("div",{"class":"fl wp49"})
        if company_detail:
            try:
                produce_company.append(company_detail[0].text)
            except:
                produce_company.append(np.NaN)
            try:
                distribute_company.append(company_detail[1].text)
            except:
                distribute_company.append(np.NaN)
    else:
        produce_company.append(np.NaN)
        distribute_company.append(np.NaN) 
dt_merged["ProductionCompany"]=produce_company
dt_merged["DistributorCompany"]=distribute_company

### Scraping finished, save the data to a CSV file

In [ ]:
dt_merged.to_csv("raw_final_data.csv")
dt_merged.head()

# Data Manipulation & Analysis

In [ ]:
### read the data
dt=pd.read_csv("raw_final_data.csv")
dt=dt.drop(dt.columns[0], axis=1)
dt.head()

## Check data types and convert to correct data types

In [ ]:
## check data types and convert to correct data types
dt.dtypes

In [ ]:
## date to date 
dt.ReleaseDate=dt.ReleaseDate.apply(lambda x: re.sub(r"日\w*","",x))
dt.ReleaseDate=dt.ReleaseDate.apply(lambda x: re.sub(r"年|月","-",x))
dt.ReleaseDate=pd.to_datetime(dt.ReleaseDate)

## Recall that different units existed in the datasets

In [ ]:
## Boxoffice/BoxofficeGlobal to float, and currency conversion, unit unified to million USD
BoxOffice=[]
BoxOfficeCN=[]
BoxOfficeNA=[]
money_local=dt.BoxOffice.apply(lambda x: float(re.sub(r"(亿USD)|(亿CNY)","",x)))
for i in range(len(dt.BoxOffice)):
    if "USD" in dt.BoxOffice[i]:
        BoxOffice.append(round(money_local[i]*100,6))
    else:
        BoxOffice.append(round(0.15*money_local[i]*100,6))
BoxOfficeGlobal=[]
for i in range(len(dt.BoxOffice)):
    x=dt.BoxOfficeGlobal[i]
    try:
        math.isnan(x)
        BoxOfficeGlobal.append(np.NaN)
    except:
        BoxOfficeGlobal.append(re.sub(r"亿USD","",x))
BoxOfficeGlobal=[round(float(i)*100,6) for i in BoxOfficeGlobal]
dt.BoxOffice=BoxOffice
dt.BoxOfficeGlobal=BoxOfficeGlobal

## For those scraped information that were original stored in the list/dictionary format
## We need to convert them back

In [ ]:
## use ast to convert str representation of lists/dictionary back to their format
AwardSummary=[]
for i in dt.AwardSummary:
    try:
        math.isnan(i)
        AwardSummary.append(np.NaN)
    except:
        AwardSummary.append(ast.literal_eval(i))
ActorAndStaff=[]
for i in dt.ActorAndStaff:
    try:
        math.isnan(i)
        ActorAndStaff.append(np.NaN)
    except:
        ActorAndStaff.append(ast.literal_eval(i))
AwardDetail=[]
for i in dt.AwardDetail:
    try:
        math.isnan(i)
        AwardDetail.append(np.NaN)
    except:
        AwardDetail.append(ast.literal_eval(i))
dt.ActorAndStaff=ActorAndStaff
dt.AwardDetail=AwardDetail
dt.AwardSummary=AwardSummary

## There is a missing value in the Runtime that can be manually corrected

In [ ]:
## manual clean: Meet the Fockers2, runtime 1h55m
dt.iloc[295,13]=dt.iloc[295,12]
dt.iloc[295,12]="115"
## Runtime to int
dt.Runtime=dt.Runtime.apply(lambda x: re.sub(r"分钟","",x))
dt=dt.astype({"Runtime":int})

## For missing values in those three rankings, change them to 0 rather than NULL

In [ ]:
dt.BoxOfficeRankCN=dt.BoxOfficeRankCN.fillna(0)
dt.BoxOfficeRankNA=dt.BoxOfficeRankNA.fillna(0)
dt.BoxOfficeRankG=dt.BoxOfficeRankG.fillna(0)
dt=dt.astype({"BoxOfficeRankCN":int,"BoxOfficeRankNA":int,"BoxOfficeRankG":int})

## For missing values in global sales, use domestic sales to represent

In [ ]:
# fill in null in BoxOfficeGlobal with values in BoxOffice
dt.BoxOfficeGlobal=dt.BoxOfficeGlobal.fillna(value=dt.BoxOffice)

## Cleaning the information in two company columns

In [ ]:
# remove extra text in companies
dt.ProductionCompany=dt.ProductionCompany.apply(lambda x: re.sub(r"制作公司 ","",x))
DistributorCompany=[]
for i in range(len(dt.DistributorCompany)):
    x=dt.DistributorCompany[i]
    try:
        math.isnan(x)
        DistributorCompany.append(np.NaN)
    except:
        DistributorCompany.append(re.sub(r"发行公司 ","",x))
dt.DistributorCompany=DistributorCompany

## Extract year & month and different genres in the data

In [ ]:
## create year & month column
dt["Year"]=dt.ReleaseDate.apply(lambda x: x.year)
dt["Month"]=dt.ReleaseDate.apply(lambda x: x.month)
## seperate genres
unique_genere=set(dt.Genre.str.split(r"/").sum())
for g in unique_genere:
    dt[g]=dt.Genre.str.contains(g,regex=False)

In [ ]:
## Check the data type again
dt.dtypes

# Visulization 
### Note that Visulization in Matplotlib does not support Chinese Characters 
### Use the following to figure out the settings

In [ ]:
## figuring out chinese output in matplot
import matplotlib
# matplotlib.matplotlib_fname()
import matplotlib.pyplot as plt
from matplotlib.font_manager import _rebuild
_rebuild()
import matplotlib.pyplot as plt
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

## Creating subsets for different areas

In [ ]:
dt_global=dt.iloc[100:200]
dt_cn=dt.iloc[0:100]
dt_na=dt.iloc[200:300]

In [ ]:
## Scatter Plots on domestic sales between China and North America
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(20,8))
ax[0][0].scatter(x="Month", y="BoxOffice",data=dt_cn)
ax[0][0].set_ylabel('China Domestic BoxOffice (Million USD)')
ax[0][0].set_xlabel('Month')
ax[0][1].scatter(x="Year", y="BoxOffice",data=dt_cn)
ax[0][1].set_ylabel('China Domestic BoxOffice (Million USD)')
ax[0][1].set_xlabel('Year')

ax[1][0].scatter(x="Month", y="BoxOffice",data=dt_na)
ax[1][0].set_ylabel('NA Domestic BoxOffice (Million USD)')
ax[1][0].set_xlabel('Month')
ax[1][1].scatter(x="Year", y="BoxOffice",data=dt_na)
ax[1][1].set_ylabel('NA Domestic BoxOffice (Million USD)')
ax[1][1].set_xlabel('Year')

## Creating monthly data for visulization

In [ ]:
Month=[]
Month_count=[]
Month_money=[]
for i in dt_cn.Month.unique():
    Month.append(i)
    Month_count.append(len(dt.Month[dt.Month==i]))
    Month_money.append(dt_cn.BoxOffice[dt_cn.Month==i].sum())
dt_cn_month=pd.DataFrame({"Month":Month,"Month_count":Month_count,"Month_money":Month_money})
dt_cn_month["Avg"]=dt_cn_month.Month_money/dt_cn_month.Month_count
dt_cn_month.to_csv("Month_data_cn.csv")
dt_cn_month=dt_cn_month.sort_values(by="Month")

dt_cn_month_sep=dt[["Month","BoxOffice"]].sort_values(by="Month")
dt_cn_month_sep_list=[]
dt_cn_month_sep_list_lab=[]
for m in dt_cn_month_sep.Month.unique():
    dt_cn_month_sep_list_lab.append(m)
    dt_cn_month_sep_list.append(dt_cn_month_sep.BoxOffice[dt_cn_month_sep.Month==m].values.tolist())
Month=[]
Month_count=[]
Month_money=[]
for i in dt_na.Month.unique():
    Month.append(i)
    Month_count.append(len(dt.Month[dt.Month==i]))
    Month_money.append(dt_na.BoxOffice[dt_na.Month==i].sum())
dt_na_month=pd.DataFrame({"Month":Month,"Month_count":Month_count,"Month_money":Month_money})
dt_na_month["Avg"]=dt_na_month.Month_money/dt_na_month.Month_count
dt_na_month.to_csv("Month_data_cn.csv")
dt_na_month=dt_na_month.sort_values(by="Month")

dt_na_month_sep=dt[["Month","BoxOffice"]].sort_values(by="Month")
dt_na_month_sep_list=[]
dt_na_month_sep_list_lab=[]
for m in dt_na_month_sep.Month.unique():
    dt_na_month_sep_list_lab.append(m)
    dt_na_month_sep_list.append(dt_na_month_sep.BoxOffice[dt_na_month_sep.Month==m].values.tolist())

In [ ]:
## show the plots for Monthly data between China & North America
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(20,12))
ax[0][0].bar(x="Month", height="Month_money",data=dt_cn_month)
ax[0][0].set_ylabel('CN Domestic BoxOffice (Million USD)')
ax[0][0].set_xlabel('Month')

ax[0][1].bar(x="Month", height="Month_money",data=dt_na_month)
ax[0][1].set_ylabel('NA GlobalBoxOffice (Million USD)')
ax[0][1].set_xlabel('Month')

ax[1][0].bar(x="Month", height="Month_count",data=dt_cn_month)
ax[1][0].set_ylabel('CN Domestic Frequency (Million USD)')
ax[1][0].set_xlabel('Month')

ax[1][1].bar(x="Month", height="Month_count",data=dt_na_month)
ax[1][1].set_ylabel('NA Domestic Frequency (Million USD)')
ax[1][1].set_xlabel('Month')

## Creating data sets for visulization of different genres

In [ ]:
### change the "dt_na" in the following to "dt_cn" and run again to get the comparison dataset for China
dt_genre_box=dt_na[["BoxOffice"]+list(unique_genere)]
for i in range(len(dt_genre_box.columns)):
    if i==0:
        continue
    for j in range(len(dt_genre_box.BoxOffice)):
        if dt_genre_box.iloc[j,i]==True:
            dt_genre_box.iloc[j,i]=dt_genre_box.BoxOffice[j]
dt_genre_box_money=dt_genre_box[dt_genre_box.columns[1:]]
Genre=[]
Money=[]
Count=[]
for i in unique_genere:
    Genre.append(i)
    Money.append(dt_genre_box_money[i].sum())
    Count.append(dt[i].sum())
dt_genre_count=pd.DataFrame({"Genre":Genre,"Count":Count,"BoxOffice":Money})
#
Genre=[]
Money=[]
for i in range(len(dt.BoxOffice)):
    for g in unique_genere:
        if g in dt.Genre[i]:
            Genre.append(g)
            Money.append(dt.BoxOffice[i])
dt_genre_count_seperate=pd.DataFrame({"Genre":Genre,"BoxOffice":Money})
dt_genre_count["Average"]=dt_genre_count.BoxOffice/dt_genre_count.Count
dt_genre_count=dt_genre_count.sort_values(by="Count")
dt_genre_count_seperate_list=[]
dt_genre_count_seperate_list_lab=[]
for g in dt_genre_count.Genre.unique():
    dt_genre_count_seperate_list_lab.append(g)
    dt_genre_count_seperate_list.append(dt_genre_count_seperate.BoxOffice[dt_genre_count_seperate.Genre==g].values.tolist())

## Creating plots for different Genres

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(20,12))
ax[0][0].bar(x="Genre", height="Count",data=dt_genre_count)
ax[0][0].set_ylabel('Frequency')
ax[0][1].bar(x="Genre", height="BoxOffice",data=dt_genre_count)
ax[0][1].set_ylabel('Total GlobalBoxOffice (Million USD)')
ax[1][0].bar(x="Genre", height="Average",data=dt_genre_count)
ax[1][0].set_ylabel('Avg GlobalBoxOffice (Million USD)')
ax[1][1].boxplot(dt_genre_count_seperate_list,flierprops=dict(marker='D', markerfacecolor='b', markersize=4,markeredgecolor='b'),
                medianprops=dict(color='r'))
ax[1][1].set_ylabel("GlobalBoxOffice (Million USD)")
ax[1][1].set_xticklabels(dt_genre_count_seperate_list_lab)

# Some exploration on the data for basic information

In [ ]:
## change "dt" to "dt_cn"/"dt_na"/"dt_global" 
## to get summary statistics for different areas
dt.describe()

## You can directly show the minmal or maximum for each data set based on sales/ratings

In [ ]:
## change "dt" to "dt_cn"/"dt_na"/"dt_global" 
## to get summary statistics for different areas
dt[dt.MtimeRating==min(dt.MtimeRating)]
dt[dt.BoxOfficeGlobal==min(dt.BoxOfficeGlobal)]

## Potential relationship between ratings & sales

In [ ]:
sns.regplot(x="MtimeRating", y="BoxOffice", data=dt)